In [1]:
import os
import csv

In [2]:
def extract_landmarks_from_result(result):
    pass

In [3]:
def create_csv_output_folder(output_dir):
    if not os.path.exists(output_dir):  # if the folder does not exist create it
        os.mkdir(output_dir)


# def create_csv_file(file_name, landmarks):
#     # if the file does not exist create it
#     if not os.path.exists(output_file_path):
#         num_coords = len(results.pose_landmarks.landmark) + \
#             len(results.face_landmarks.landmark)
#         landmarks = []
#         landmarks += ['timestamp']
#         landmarks += ['class']

#         for val in range(1, num_coords+1):
#             landmarks += ['x{}'.format(val), 'y{}'.format(val),
#                           'z{}'.format(val), 'v{}'.format(val)]

#     with open(file_name, mode='w', newline='') as f:
#         csv_writer = csv.writer(
#             f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#         csv_writer.writerow(landmarks)

def create_csv_output_file(output_file, results):
    current_dir = os.path.dirname(os.path.abspath(__file__))
    output_dir = os.path.join(current_dir, 'csv_output_folder')

    create_csv_output_folder(output_dir) # creating the output folder

    output_file = output_dir + '/' + output_file
    create_csv_file(output_file, landmarks) # creating the output file

In [1]:
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mediapipe as mp
import pandas as pd
import numpy as np
import csv


def create_csv_file(file):
    if not os.path.isfile(file):
        header = ['label']
        for i in range(1, 544):
            header += [f'x{i}', f'y{i}', f'z{i}', f'v{i}']
        with open(file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(header)


def extract_landmarks(landmarks):
    if landmarks:
        return list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in landmarks.landmark]).flatten())
    else:
        return [0] * 84


def process_video(input_file, output_file, gesture):
    
    create_csv_file(output_file) #create the file if it does not exist

    mp_holistic, mp_drawing = mp.solutions.holistic, mp.solutions.drawing_utils
    with mp_holistic.Holistic(static_image_mode=True) as holistic:
        cap = cv2.VideoCapture(input_file)
        coords = []
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, _ = image.shape
            results = holistic.process(image)

            # Extract landmarks for each body part
            face_row = extract_landmarks(results.face_landmarks)
            left_hand_row = extract_landmarks(results.left_hand_landmarks)
            right_hand_row = extract_landmarks(results.right_hand_landmarks)
            pose_row = extract_landmarks(results.pose_landmarks)

            # Combine all landmark points into a single list
            feature_vector = face_row + pose_row + left_hand_row + right_hand_row
            if results.face_landmarks:
                feature_vector = [gesture] + feature_vector
                coords.append(feature_vector)
        cap.release()
        df = pd.DataFrame(coords)
        df.to_csv(output_file, mode='a', index=False, header=False)
        return df

In [10]:
import os

train_size = 2 # how many videos of each gesture the test test will include

directory = os.getcwd() + '/videos'

folders = os.listdir(directory)

for folder in folders:
    path = directory + '/' + folder
    files = os.listdir(path)
    print('folder ' + folder + ' has ' + str(len(files)) + ' videos ')

folder angry has 19 videos 
folder bored has 33 videos 
folder disgust has 13 videos 
folder happy has 25 videos 
folder sad has 29 videos 
folder shy has 17 videos 
folder stressed has 22 videos 
folder surprised has 22 videos 


In [11]:
import os

train_size = 3 # how many videos of each gesture the test test will include

directory = os.getcwd() + '/videos'

folders = os.listdir(directory)

counter_folder = 1
for folder in folders:
    counter_file = 1
    path = directory + '/' + folder
    files = os.listdir(path)
    for i, file in enumerate(files):
        print('folder ' + str(counter_folder)+'/'+str(len(folders)) + ', file ' + str(counter_file) + '/' + str(len(files)))

        if i < len(files)-train_size:
            process_video(path + '/' + file, 'train_set4.csv', folder)
        else:
            process_video(path + '/' + file, 'test_set4.csv', folder)

        counter_file+=1
    counter_folder+=1

folder 1/8, file 1/19
folder 1/8, file 2/19
folder 1/8, file 3/19
folder 1/8, file 4/19
folder 1/8, file 5/19
folder 1/8, file 6/19
folder 1/8, file 7/19
folder 1/8, file 8/19
folder 1/8, file 9/19
folder 1/8, file 10/19
folder 1/8, file 11/19
folder 1/8, file 12/19
folder 1/8, file 13/19
folder 1/8, file 14/19
folder 1/8, file 15/19
folder 1/8, file 16/19
folder 1/8, file 17/19
folder 1/8, file 18/19
folder 1/8, file 19/19
folder 2/8, file 1/33
folder 2/8, file 2/33
folder 2/8, file 3/33
folder 2/8, file 4/33
folder 2/8, file 5/33
folder 2/8, file 6/33
folder 2/8, file 7/33
folder 2/8, file 8/33
folder 2/8, file 9/33
folder 2/8, file 10/33
folder 2/8, file 11/33
folder 2/8, file 12/33
folder 2/8, file 13/33
folder 2/8, file 14/33
folder 2/8, file 15/33
folder 2/8, file 16/33
folder 2/8, file 17/33
folder 2/8, file 18/33
folder 2/8, file 19/33
folder 2/8, file 20/33
folder 2/8, file 21/33
folder 2/8, file 22/33
folder 2/8, file 23/33
folder 2/8, file 24/33
folder 2/8, file 25/33
folder

In [4]:
# adding more coordinates to the csv sets

import os

gesture = 'angry'

directory = os.getcwd() + '/videos/add_videos'

files = os.listdir(directory)

counter_file = 1
for file in files:
    path = directory + '/' + file
    print('file ' + str(counter_file) + ' / ' + str(len(files)))
    process_video(path, 'sets/train_set4.csv', gesture)
    counter_file+=1

file 1 / 22
file 2 / 22
file 3 / 22
file 4 / 22
file 5 / 22
file 6 / 22
file 7 / 22
file 8 / 22
file 9 / 22
file 10 / 22
file 11 / 22
file 12 / 22
file 13 / 22
file 14 / 22
file 15 / 22
file 16 / 22
file 17 / 22
file 18 / 22
file 19 / 22
file 20 / 22
file 21 / 22
file 22 / 22


In [14]:
os.path.isfile('set.csv')

True

In [ ]:
def predict(input_file, model):
    mp_holistic = mp.solutions.holistic
    with mp_holistic.Holistic(static_image_mode=True) as holistic:
        cap = cv2.VideoCapture(input_file)
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, _ = image.shape
            results = holistic.process(image)

            # Extract landmarks for each body part
            face_row = extract_landmarks(results.face_landmarks)
            left_hand_row = extract_landmarks(results.left_hand_landmarks)
            right_hand_row = extract_landmarks(results.right_hand_landmarks)
            pose_row = extract_landmarks(results.pose_landmarks)

            # Combine all landmark points into a single list
            feature_vector = face_row + pose_row + left_hand_row + right_hand_row
            if results.face_landmarks:
                # Reshape feature vector to match the input shape of the model
                feature_vector = np.array(feature_vector).reshape(1, -1)
                # Normalize the feature vector
                feature_vector = feature_vector / \
                    np.linalg.norm(feature_vector)
                # Make a prediction using the model
                prediction = model.predict(feature_vector)[0]
                # Display the prediction on the image
                cv2.putText(image, prediction, (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # Show the image
            cv2.imshow('Video', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


In [13]:
# used to create a train set csv of coordinates
# reads a video file mp4 and add its coordinates to a csv file
# input_file: a video of a specific gesture of type mp4, 
# output_file: a CSV file containing the co-ordinates with the specified gesture

def f(input_file, output_file):
    pass

In [1]:
import numpy as np
import cv2
import mediapipe as mp
import numpy as np
from tensorflow import keras

def extract_landmarks(landmarks):
    if landmarks:
        return list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in landmarks.landmark]).flatten())
    else:
        return [0] * 84

def test_model():
    
    mp_holistic, mp_drawing = mp.solutions.holistic, mp.solutions.drawing_utils
    model = keras.models.load_model(model_path)

    with mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        cap = cv2.VideoCapture(0)

        while True:
            success, image = cap.read()
            if not success:
                break

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, _ = image.shape

            results = holistic.process(image)

            face_row = extract_landmarks(results.face_landmarks)
            left_hand_row = extract_landmarks(results.left_hand_landmarks)
            right_hand_row = extract_landmarks(results.right_hand_landmarks)
            pose_row = extract_landmarks(results.pose_landmarks)

            feature_vector = face_row + pose_row + left_hand_row + right_hand_row

            if results.face_landmarks:
                feature_vector = np.array(feature_vector).reshape((1, -1))
                prediction = model.predict(feature_vector)
                gesture = get_gesture_name(prediction)

                image = cv2.putText(image, gesture + ' {:.2f}'.format(max(prediction.flatten())), (10, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            cv2.imshow('Holistic Model', image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


def get_gesture_name(prediction):

    gesture_names = ['angry', 'bored', 'disgust', 'happy', 'sad', 'shy', 'stressed', 'surprised']

    index = np.argmax(prediction)

    return gesture_names[index]

In [2]:
model_path = 'models/my_model5.h5'
test_model()

1/1 [==============================] - 0s 26ms/step
